<a href="https://colab.research.google.com/github/davidpagnon/Sports2D/blob/main/Sports2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Sports2D in browser

**[Sports2D](https://github.com/davidpagnon/Sports2D)** offers a way to compute 2D joint and segment angles from a video.\
This notebook lets you benefit from its full capacities, even if you cannot install [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) on your device.
You can batch-process several videos at the same time, check the results, and download them on your device.

</br>

To use it, run each cell one after another.\
**Check that you are running a GPU: "Runtime" tab, click "change runtime type"**


<img src="https://raw.githubusercontent.com/davidpagnon/Sports2D/main/Content/demo_gif.gif" title='Demonstration of Sports2D with OpenPose.'  width="760">

`Warning:` Your data will be sent to the Google servers, which do not follow the European GDPR requirements regarding privacy.\
`Warning:` OpenPose installation can last about 30 minutes, and needs to be done anew each time you disconnect from this Colab notebook. Moreover, the server disconnects after 90 minutes of idle time.

##### Known issues





1. OpenPose may (sometimes, not always) make the server run out of memory `CudaSuccess (2 vs. 0) out of memory` error. Reasons may be:
   1. OpenPose 1.7 does not correctly take advantage of CuDNN -> Try to downgrade to OpenPose 1.6 
   2. The Colab current versions of CuDNN and CUDA are incompatible (`CUDNN_STATUS_VERSION_MISMATCH`). CuDNN v8.7.0 and CUDA v11.8, vs. CUDA v11.2 tested with WindowsPortableDemo.exe.

2. Videos generated by Sports2D (i.e., by OpenCV and `cv2.VideoWriter_fourcc(*'mp4v'))` are perfectly read once downloaded, but cannot be read in Colab by IPython.display. It may be that 'h264' should be preferred, but `cv2.VideoWriter_fourcc(*'h264'))`` does not work on my computer. Maybe it would be worth to convert them with ffmpeg, or to find an other way to display them. 

I would be grateful for some help!

In [ ]:
# First check that CUDA is preinstalled, and check CuDNN version
! nvcc --version
print('\n')
!ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so.*
print('\n')
# Check that you are running a GPU
# If not, select "change runtime type" from "Runtime" tab
! nvidia-smi
print('\n')
# Ubuntu version
!cat /etc/*release

In [ ]:
# 1. Try downgrading OpenPose to v1.6.0 ?

import os
from os.path import exists, join, basename, splitext

%rm -rf openpose
ver_openpose = "v1.6.0"
git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake because of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose, version 1.6.0 because 1.7 uses too much memory and does not take advantage of cudnn
  !git clone -b "$ver_openpose" $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`  

In [ ]:
# 2. Try to change CUDA and CuDNN versions 
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
# 2. Try to downgrade CUDA

#Uninstall the current CUDA version
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

# Install cuda 11.2
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
!mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.2.0/local_installers/cuda-repo-ubuntu2004-11-2-local_11.2.0-460.27.04-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2004-11-2-local_11.2.0-460.27.04-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu2004-11-2-local/7fa2af80.pub
!apt-get update
!apt-get -y install cuda-11.2


# Installation

## Install Sports2D

In [ ]:
# install the right version of python
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1

# install Sports2D
!pip install sports2D

## Install OpenPose

Adapted from [there](https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/OpenPose.ipynb). This can last about 20 minutes.\
Skip this whole section if you are happy with working with BlazePose instead of OpenPose (single person, slightly less accurate).


In [ ]:
import os
from os.path import exists, join, basename, splitext

%rm -rf openpose
git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake because of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  !git clone $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`  

Install BODY_25B model

In [ ]:
!wget posefs1.perception.cs.cmu.edu/OpenPose/models/pose/1_25BBkg/body_25b/pose_iter_XXXXXX.caffemodel -P /content/openpose/models/pose/body_25b
!wget https://raw.githubusercontent.com/CMU-Perceptual-Computing-Lab/openpose_train/master/experimental_models/1_25BBkg/body_25b/pose_deploy.prototxt -P /content/openpose/models/pose/body_25b

# Run Sports2D


## Upload videos

Select the videos you want to process.\
Warning: Don't forget to click on the button "select file" below the code!

In [ ]:
from google.colab import files

!mkdir -p /content/videos
%cd /content/videos
uploaded = files.upload()

## Run Sports2D

Batch run all uploaded videos with Sports2D

In [51]:
import toml

# Retrieve Sports2D Config and video files
!cp /usr/local/lib/python3.10/dist-packages/Sports2D/Demo/Config_demo.toml /content/videos/Config.toml
!cp /usr/local/lib/python3.10/dist-packages/Sports2D/Demo/demo.mp4 /content/videos/demo.mp4
config_dict = toml.load('/content/videos/Config.toml')

# If you want to change your analysis parameters, do it now by hand by editing '/content/videos/Config.toml'
# or do it like so:
config_dict.get('pose').update({'pose_algo':'OPENPOSE'})

In [52]:
# Batch run all videos with Sports2D
# The video_file item will be updated automatically with each of the videos you uploaded
from Sports2D import Sports2D
for vid in uploaded.keys():
  config_dict.get('project').update({'video_file':vid})
  with open('Config.toml', "w") as config_file: toml.dump(config_dict, config_file)
  Sports2D.detect_pose('Config.toml')
  Sports2D.compute_angles('Config.toml')

## Show results

Show video

In [60]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

In [ ]:
# The video might not be displayed as openCV encodes it in 'mp4v' (and not 'h364', for example). 
# You can still download it and play it seamlessly on your device
show_local_mp4_video('/content/videos/demo_BODY_25B.mp4', width=960, height=720)

Plot angles

In [ ]:
# Retrieve angle results
import pandas as pd
angles = pd.read_csv('/content/videos/demo_BODY_25B_person1_angles.csv', header=[0,1,2,3])
angles

In [ ]:
# Plot right and left knee angles
angles['DavidPagnon','person0'][['Right knee', 'Left knee']].plot()

## Download results

In [ ]:
%cd /content

# Compress result folder
!zip -r Sports2D_results.zip videos

# Download zip file
files.download('Sports2D_results.zip')